In [4]:
import numpy as np
import pandas as pd
from timestamp import string2timestamp

In [2]:
class Dataset_constructor(object):

    def __init__(self, data, timestamps, T=48, CheckComplete=True):
        super(Dataset_constructor, self).__init__()
        assert len(data) == len(timestamps)
        self.data = data
        self.timestamps = timestamps# [b'2013070101', b'2013070102']
        self.T = T
        self.pd_timestamps = string2timestamp(timestamps, T=self.T)
        self.make_index()  # 将时间戳：做成一个字典，也就是给每个时间戳一个序号

    def make_index(self):
        self.get_index = dict()
        for i, ts in enumerate(self.pd_timestamps):
            self.get_index[ts] = i
            
    def get_matrix(self, timestamp):  # 给定时间戳返回对于的数据
        return self.data[self.get_index[timestamp]]

    def check_it(self, depends):
        for d in depends:
            if d not in self.get_index.keys():
                return False
        return True

    def create_dataset(self, c_length=3, t_length=3, t_inter=7, p_length=3, p_inter=1):
        offset_frame = pd.DateOffset(minutes=24 * 60 // self.T)  # 时间偏移 minutes = 30
        print(123)
        C_list = []
        P_list = []
        T_list = []
        Y_list = []
        timestamps_Y = []
        time_range = [range(1, c_length + 1),
                      [47,48,49],
#                       [p_inter * self.T * j-1,p_inter * self.T * j,p_inter * self.T * j+1 for j in range(1, p_length +1)],
                      [t_inter * self.T * j for j in range(1, t_length + 1)]]

        # print depends # [range(1, 4), [48, 96, 144], [336, 672, 1008]]
        bound = max(self.T * t_inter * t_length, self.T * p_inter * p_length+1, c_length)
        while bound < len(self.pd_timestamps):
            Flag = True
            for branch in time_range:
                if Flag is False:
                    break
                Flag = self.check_it([self.pd_timestamps[bound] - j * offset_frame for j in branch])

            if Flag is False:
                bound += 1
                continue
            x_c = [self.get_matrix(self.pd_timestamps[bound] - j * offset_frame) for j in time_range[0]]
            print(self.pd_timestamps[bound])
            print(adshdj)
            # 取当前时刻的前3个时间片的数据数据构成“邻近性”模块中一个输入序列
            # 例如当前时刻为[Timestamp('2013-07-01 00:00:00')]
            # 则取：
            # [Timestamp('2013-06-30 23:30:00'), Timestamp('2013-06-30 23:00:00'), Timestamp('2013-06-30 22:30:00')]
            #  三个时刻所对应的in-out flow为一个序列
            x_p = [self.get_matrix(self.pd_timestamps[bound] - j * offset_frame) for j in time_range[1]]
            # 取当前时刻 前 1*PeriodInterval,2*PeriodInterval,...,len_period*PeriodInterval
            # 天对应时刻的in-out flow 作为一个序列，例如按默认值为 取前1、2、3天同一时刻的In-out flow
            print(time_range[2])
            x_t = [self.get_matrix(self.pd_timestamps[bound] - j * offset_frame) for j in time_range[2]]
            # 取当前时刻 前 1*TrendInterval,2*TrendInterval,...,len_trend*TrendInterval
            # 天对应时刻的in-out flow 作为一个序列,例如按默认值为 取 前7、14、21天同一时刻的In-out flow
            y = self.get_matrix(self.pd_timestamps[bound])
            if c_length > 0:
                C_list.append(np.vstack(x_c))
                # a.shape=[2,32,32] b.shape=[2,32,32] c=np.vstack((a,b)) -->c.shape = [4,32,32]
            if p_length > 0:
                P_list.append(np.vstack(x_p))
            if t_length > 0:
                T_list.append(np.vstack(x_t))
            Y_list.append(y)
            timestamps_Y.append(self.timestamps[bound])
        
            bound += 1
        XC = np.asarray(C_list)  # 模拟 邻近性的 数据 [?,6,32,32]
        XP = np.asarray(P_list)  # 模拟 周期性的 数据 隔天
        XT = np.asarray(T_list)  # 模拟 趋势性的 数据 隔周
        Y = np.asarray(Y_list)# [?,2,32,32]
        print(12332141242)
        print("XC shape: ", XC.shape, "XP shape: ", XP.shape, "XT shape: ", XT.shape, "Y shape:", Y.shape)
        return XC, XP, XT, Y, timestamps_Y